# Semantic functions

### Initialize the kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.18.0-rc"

#!import ../../Secrets.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(
        deploymentName: Secrets.DeploymentName,
        endpoint: Secrets.Endpoint,
        apiKey: Secrets.ApyKey)
    // .AddOpenAIChatCompletion(
    //     modelId: Secrets.OpenAIModel,
    //     apiKey: Secrets.OpenAIApiKey)
    .Build();

### ✍️ Create a simple plain semantic function

In [ ]:
var promptTemplate = "When is the next total solar eclipse?";

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate);
var response = await kernel.InvokeAsync(semanticFunction);

display(response.GetValue<string>())

### 📋 Now let's create a more complete semantic function (with input variable and execution settings)

In [ ]:
var promptTemplate = "When is the next total {{$input}} eclipse? Respond only with JSON.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
    MaxTokens = 100,
    Temperature = 0.1,
};

var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate, executionSettings);

var kernelArgument = new KernelArguments()
{
    ["input"] = "lunar"
};

var response = await kernel.InvokeAsync(semanticFunction, kernelArgument);

display(response.GetValue<string>())

### 📋 But what about if we have more input variables?
### 🪴 This simple bit of "grounding" reduces so-called 'hallucination' with LLMs

In [ ]:
var promptTemplate = "When is the next total {{$input}} eclipse? Today is {{$today}}. Respond only with JSON.";

var executionSettings = new OpenAIPromptExecutionSettings 
{
     MaxTokens = 500,
     Temperature = 0.1
};
var semanticFunction = kernel.CreateFunctionFromPrompt(promptTemplate, executionSettings);

var kernelArguments = new KernelArguments(executionSettings)
{
    ["input"] = "solar",
    ["today"] = "15th of June 2024"
};

var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);

display(response)